In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import tensorrt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import LSTM, Dense

In [ ]:
raw_df = pd.read_csv('car_data.csv')
print(raw_df.shape)
raw_df.head(100)

In [ ]:
def SoC_forecasting_model(timesteps, features, batch_size, output_shape, dropout=0, recurrent_dropout=0, verbose=True):
    """
    cuDNN Requirements
    activation == tanh
    recurrent_activation == sigmoid
    recurrent_dropout == 0
    unroll is False
    use_bias is True
    """
    STATEFUL = True
    SoC_model = Sequential(name ='SoC_Forecasting_Model')
    SoC_model.add(LSTM(16, input_shape=(timesteps, features), batch_size=batch_size, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True, stateful=STATEFUL, name='LSTM_1'))

    SoC_model.add(LSTM(32, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True, stateful=STATEFUL, name='LSTM_2'))
    #SoC_model.add(LSTM(16, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True, stateful=STATEFUL, name='LSTM_hidden2'))


    SoC_model.add(LSTM(16, dropout=dropout, recurrent_dropout=recurrent_dropout, stateful=STATEFUL, name='LSTM_last'))
    SoC_model.add(Dense(output_shape, name='Dense_output'))
    SoC_model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    if verbose :
        print(SoC_model.summary())
    return SoC_model


In [ ]:
def AIO_preprocess_train_SoC_model(df, look_back=32, look_ahead=1, test_size=0.2, epochs=50, batch_size=256, verbose=True, evaluate=True):

    scaler_X = MinMaxScaler()
    scaler_Y = MinMaxScaler()

    df.astype(float)

    #df['voltage_smoothed'] = df['voltage']
    df['voltage_smoothed'] = df['voltage'].ewm(alpha=0.1, adjust=False).mean()
    df['voltage_smoothed'] = df['voltage_smoothed'].fillna(df['voltage'])


    raw_X = df.drop(['timestamp', 'voltage'], axis=1)

    raw_Y = np.array(df['voltage_smoothed'])

    raw_X = scaler_X.fit_transform(raw_X)
    raw_Y = scaler_Y.fit_transform(raw_Y.reshape(-1,1))
    raw_Y = raw_Y.reshape(-1)

    X_train, X_test, Y_train, Y_test = train_test_split(np.array(raw_X), np.array(raw_Y), test_size=test_size, shuffle=False)
    model = SoC_forecasting_model(look_back, X_train.shape[1], batch_size, look_ahead, dropout=0, verbose=verbose)
    if verbose :
        print('X_train  :',X_train.shape)
        print('Y_train  :',Y_train.shape)
        print('X_test   :',X_test.shape)
        print('Y_test   :',Y_test.shape)

    mse_train = []
    mae_train = []
    mse_test = []
    mae_test = []
    if verbose :
        print(f'{"TRAINING":-^100}')
    for i in range(epochs):
        for j in tqdm(range((X_train.shape[0]-look_ahead-look_back)//batch_size), ncols=100, desc=f"Epoch: {i+1:0>{len(str(epochs))}}/{epochs:0>{len(str(epochs))}}", colour='green', disable=not verbose):
            X_batch = np.empty((batch_size, look_back, X_train.shape[1]))
            Y_batch = np.empty((batch_size, look_ahead))
            for k in range(j*batch_size, (j+1)*batch_size):
                X_batch[k - j*batch_size] = X_train[k : k + look_back]
                Y_batch[k - j*batch_size] = Y_train[k + look_back : k + look_back + look_ahead]
            metrics = model.train_on_batch(X_batch, Y_batch, reset_metrics=False)
            mse_train.append(metrics[0])
            mae_train.append(metrics[1])
        model.reset_states()

    if verbose :
        plt.plot(mse_train, label='mse')
        plt.plot(mae_train, label='mae')
        plt.title('Training stats')
        plt.legend()
        plt.show()

    if evaluate :
        if verbose :
            print(f'{"EVALUATION":-^100}')
        for j in tqdm(range((X_test.shape[0]-look_ahead-look_back)//batch_size), ncols=100, desc='Evaluation', colour='red', disable=not verbose):
            X_batch = np.empty((batch_size, look_back, X_test.shape[1]))
            Y_batch = np.empty((batch_size, look_ahead))
            for k in range(j*batch_size, (j+1)*batch_size):
                X_batch[k - j*batch_size] = X_test[k : k + look_back]
                Y_batch[k - j*batch_size] = Y_test[k + look_back : k + look_back + look_ahead]
            metrics = model.test_on_batch(X_batch, Y_batch)
            mse_test.append(metrics[0])
            mae_test.append(metrics[1])
        if verbose :
            plt.plot(mse_test, label='mse')
            plt.plot(mae_test, label='mae')
            plt.title('Testing stats')
            plt.legend()
            plt.show()

    return model, scaler_X, scaler_Y, X_train, X_test, Y_train, Y_test

model, scaler_X, scaler_Y, X_train, X_test, Y_train, Y_test = AIO_preprocess_train_SoC_model(raw_df)

In [ ]:
res =[]
look_back=32
look_ahead=1
batch_size=256
for j in tqdm(range((X_test.shape[0]-look_back-look_ahead)//batch_size), ncols=100, desc='Prediction', colour='blue'):
    X_batch = np.empty((batch_size, look_back, X_test.shape[1]))
    for k in range(j*batch_size, (j+1)*batch_size):
        X_batch[k - j*batch_size] = X_test[k : k + look_back]
    Y_batch = model.predict_on_batch(X_batch)
    res.extend(Y_batch.reshape(-1))
plt.plot(res, label='prediction')
plt.plot(Y_test, label='real')
plt.legend()
plt.show()
